# Projeto 2 - Ciência dos Dados

Nome: André Luís Silva Lopes

Nome: João Gianfaldoni Andrade

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [59]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [60]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from nltk.corpus import stopwords
stopWords = set(stopwords.words('portuguese'))

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [61]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @JoaoAnd93050623

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [62]:
#Produto escolhido:
produto = 'SPTrans'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [34]:
#Cria um objeto para a captura
api = tweepy.API(auth)


#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text) and msg.author.screen_name != "sptrans": 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [63]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [98]:
SPTrans = pd.read_excel('SPTrans.xlsx')
SPTrans_I = SPTrans[SPTrans.Classificacao == "I"]
SPTrans_R = SPTrans[SPTrans.Classificacao == "R"]
SPTrans_R = SPTrans_R.reset_index().drop(columns = ["index"])
SPTrans_I = SPTrans_I.reset_index().drop(columns = ["index"])
Irrelevantes_raw = []
Relevantes_raw = []

In [99]:
for i in range(0, len(SPTrans_R.Treinamento)):
    Relevantes_raw.append(SPTrans_R.Treinamento[i])
for i in range(0, len(SPTrans_I.Treinamento)):
    Irrelevantes_raw.append(SPTrans_I.Treinamento[i])

In [107]:
import re 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    #text = ' '.join(word for word in text.split() if word.lower() not in stopWords)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed  

In [101]:
Relevantes_raw_txt = ', '.join(Relevantes_raw)
Irrelevantes_raw_txt = ','.join(str(v) for v in Irrelevantes_raw)
Relevantes = cleanup(Relevantes_raw_txt.lower())
Irrelevantes = cleanup(Irrelevantes_raw_txt.lower())

In [102]:
Serie_Relevantes  = pd.Series(Relevantes.split())
Serie_Irrelevantes  = pd.Series(Irrelevantes.split())
Tabela_Irrelevantes = Serie_Irrelevantes.value_counts(True)
Tabela_Relevantes = Serie_Relevantes.value_counts(True)
Palavras = Relevantes + Irrelevantes

In [103]:
SPTrans_teste = pd.read_excel('SPTrans.xlsx', "Teste")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [94]:
Tweet = "A SPTrans e muito ruim e eu essa merda de empresa ar condicionado"
Tweet = cleanup(Tweet.lower())
P_Relevante = 1
P_Irrelevante = 1
for palavra in Tweet.split():
    if palavra in Tabela_Relevantes:
        P_Relevante *= (Tabela_Relevantes[palavra]+ 0.001)
    else:
        P_Relevante *= 0.001
    if palavra in Tabela_Irrelevantes:
        P_Irrelevante *= (Tabela_Irrelevantes[palavra] + 0.001)
    else:
        P_Irrelevante *= 0.001

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [108]:
P_Relevante = 1
P_Irrelevante = 1
A = 0
T = 0
for i in range (0, len(SPTrans_teste.Teste)):
    P_Relevante = 1
    P_Irrelevante = 1
    Tweet = SPTrans_teste.Teste[i]
    Tweet = cleanup(Tweet.lower())
    for palavra in Tweet.split():
        if palavra in Tabela_Relevantes:
            P_Relevante *= (Tabela_Relevantes[palavra]+ 0.001)
        else:
            P_Relevante *= 0.001
        if palavra in Tabela_Irrelevantes:
            P_Irrelevante *= (Tabela_Irrelevantes[palavra] + 0.001)
        else:
            P_Irrelevante *= 0.001
    if P_Relevante > P_Irrelevante and SPTrans_teste.Classificacao[i] == "R":
        A+=1
        T+=1
    elif P_Irrelevante > P_Relevante and SPTrans_teste.Classificacao[i] == "I":
        A+=1
        T+=1
    else:
        T+=1
print(A/T)
print(A)

0.655
131


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**